# Segmenting and Clustering Neighborhoods

+ beautifulsoup get web table
+ pandas get data as process
+ 

## import package

In [ ]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## get postal data

In [39]:
# get postal code data with beautifulsoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

with open("data/postal_codes_sp.csv","w") as fp:
    title = "Postcode,Borough,Neighbourhood"
    tables = soup.findAll('table')
    tab = tables[0]
    
    fp.write(title)
    for tr in tab.tbody.findAll('tr'):
        row = []
        for td in tr.findAll('td'):
            row.append(td.get_text().replace("\n", ""))
        fp.write(",".join(row))
        fp.write('\n')

## data process by pandas

In [40]:
df = pd.read_csv("data/postal_codes_sp.csv")
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [41]:
# drop "Not assigned" Borough and set "Not assigned" Neighbourhood
df = df[~df['Borough'].isin(["Not assigned"])]
df["Neighbourhood"][df['Neighbourhood']=="Not assigned"] = df["Borough"][df['Neighbourhood']=="Not assigned"]
df.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [42]:
# join Neighbourhood for same Postcode and Borough
df_group = df.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x: ','.join(x))
df_group = df_group.reset_index()
df_group.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [43]:
df_group.shape

(103, 3)